In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
from gdrive.MyDrive.trading.src.lib import *
from gdrive.MyDrive.trading.src.sampler import *
from gdrive.MyDrive.trading.src.agents import *
from gdrive.MyDrive.trading.src.emulator import *
from gdrive.MyDrive.trading.src.simulators import *
from gdrive.MyDrive.trading.src.visualizer import *

In [ ]:

def get_model(model_type, env, learning_rate, fld_load):

	print_t = False
	exploration_init = 1.

	if model_type == 'MLP':
		m = 16
		layers = 5
		hidden_size = [m]*layers
		model = QModelMLP(env.state_shape, env.n_action)                               ## 
		model.build_model(hidden_size, learning_rate=learning_rate, activation='tanh') ## construye el MLP
	
	elif model_type == 'conv':

		m = 16
		layers = 2
		filter_num = [m]*layers
		filter_size = [3] * len(filter_num)
		#use_pool = [False, True, False, True]
		#use_pool = [False, False, True, False, False, True]
		use_pool = None
		#dilation = [1,2,4,8]
		dilation = None
		dense_units = [48,24]
		model = QModelConv(env.state_shape, env.n_action)
		model.build_model(filter_num, filter_size, dense_units, learning_rate, 
			dilation=dilation, use_pool=use_pool)

	elif model_type == 'GRU':

		m = 32
		layers = 3
		hidden_size = [m]*layers
		dense_units = [m,m]
		model = QModelGRU(env.state_shape, env.n_action)
		model.build_model(hidden_size, dense_units, learning_rate=learning_rate)
		print_t = True

	elif model_type == 'ConvRNN':
	
		m = 8
		conv_n_hidden = [m,m]
		RNN_n_hidden = [m,m]
		dense_units = [m,m]
		model = QModelConvGRU(env.state_shape, env.n_action)
		model.build_model(conv_n_hidden, RNN_n_hidden, dense_units, learning_rate=learning_rate)
		print_t = True

	elif model_type == 'pretrained':
		agent.model = load_model(fld_load, learning_rate)

	else:
		raise ValueError
		
	return model, print_t


In [ ]:
model_type = 'conv'; exploration_init = 1.; fld_load = None
n_episode_training = 100
n_episode_testing = 100
open_cost = 3.3
#db_type = 'SinSamplerDB'; db = 'concat_half_base_'; Sampler = SinSampler
db_type = 'PairSamplerDB'; db = 'randjump_100,1(10, 30)[]_'; Sampler = PairSampler
batch_size = 8
learning_rate = 1e-4
discount_factor = 0.8
exploration_decay = 0.99
exploration_min = 0.01
window_state = 40

fld = os.path.join('gdrive/MyDrive/trading','data',db_type,db+'A')
sampler = Sampler('load', fld=fld)
env = Market(sampler, window_state, open_cost)
model, print_t = get_model(model_type, env, learning_rate, fld_load)
model.model.summary()
#return




Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 40, 2)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 38, 16)            112       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 19, 16)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 17, 16)            784       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 8, 16)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 48)               

In [ ]:

agent = Agent(model, discount_factor=discount_factor, batch_size=batch_size)
visualizer = Visualizer(env.action_labels)


OUTPUT_FLD = 'gdrive/MyDrive/trading/results'
fld_save = os.path.join(OUTPUT_FLD, sampler.title, model.model_name, 
  str((env.window_state, sampler.window_episode, agent.batch_size, learning_rate, agent.discount_factor, exploration_decay, env.open_cost)))
	
print('='*20)
print(fld_save)
print('='*20)
  

simulator = Simulator(agent, env, visualizer=visualizer, fld_save=fld_save)
x=simulator.train(n_episode_training, save_per_episode=1, exploration_decay=exploration_decay, exploration_min=exploration_min, print_t=print_t, exploration_init=exploration_init)
	#agent.model = load_model(os.path.join(fld_save,'model'), learning_rate)

	#print('='*20+'\nin-sample testing\n'+'='*20)
y=simulator.test(n_episode_testing, save_per_episode=1, subfld='in-sample testing')


gdrive/MyDrive/trading/results/DB_randjump(5, (10, 30), 1, [])/Conv[(16, 3, 1, True), (16, 3, 1, True)] + [48, 24]/(40, 180, 8, 0.0001, 0.8, 0.99, 3.3)

training...
0	[0]_forecast_horizon=[10]	99.0	-195.5	-5.6	-195.5	-5.6
saving results...

training...
1	[1]_forecast_horizon=[23]	98.0	-300.2	0.0	-247.9	-2.8
saving results...

training...
2	[2]_forecast_horizon=[23]	97.0	-148.8	0.0	-195.5	0.0
saving results...

training...
3	[3]_forecast_horizon=[29]	96.1	-256.9	0.0	-226.2	0.0
saving results...

training...
4	[4]_forecast_horizon=[25]	95.1	-240.2	1.0	-240.2	0.0
saving results...

training...
5	[5]_forecast_horizon=[26]	94.1	-487.3	-13.5	-248.5	0.0
saving results...

training...
6	[6]_forecast_horizon=[29]	93.2	-334.9	87.6	-256.9	0.0
saving results...

training...
7	[7]_forecast_horizon=[10]	92.3	-141.4	7.7	-248.5	0.0
saving results...

training...
8	[8]_forecast_horizon=[23]	91.4	-395.1	-16.7	-256.9	0.0
saving results...

training...
9	[9]_forecast_horizon=[15]	90.4	-525.6	0.0	-278.5	0.